In [3]:
import torch
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR10        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import detectors
import timm
from src.temperature_scaling import ModelWithTemperature
from src.saps_hyp_opt import lambda_optimization_saps

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet34_cifar10", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])  # normalize
])
# load images from CIFAR10
dataset = CIFAR10(root="../../../data", train=False, download=True, transform=data_transform)

# temperature scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=5.0).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

lambda_values = [0, 0.01, 0.05, 0.07, 0.1, 0.2, 0.5, 1.0, 1.5, 2.0, 2.2, 2.5]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_saps(model, dataset, lambda_values, device=device)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")

Files already downloaded and verified
Before temperature - NLL: 0.327, ECE: 0.046
Optimal temperature: 4.904
After temperature - NLL: 0.882, ECE: 0.466
Looking for optimal lambda...
Optimal lambda is 2.5

